# First Method of Simulated Moments (MSM) estimation with estimagic

This tutorial shows how to do a Method of Simulated Moments estimation in estimagic. The Method of Simulated Moments (MSM) is a nonlinear estimation principle that is very useful to fit complicated models to data. The only thing that is needed is a function that simulates model outcomes that you observe in some empirical dataset. 

The tutorial uses an example model from [Rick Evans'](https://github.com/rickecon/StructEst_W20) great tutorial on MSM. The model is deliberately simple so we can focus on the mechanics of the MSM estimation. 

Throughout the model we only talk about MSM estimation, however, the more general case of indirect inference estimation works exactly the same way. 


## The steps of MSM estimation

- load empirical data 
- define a function calculate estimation moments on the data 
- calculate the covariance matrix of the empirical moments (with ``get_moments_cov``)
- define a function to simulate moments from the model 
- estimate the model, calculate standard errors, do sensitivity analysis (with ``estimate_msm``)

## The example model 

The model whose parameters we estimate is a truncated normal distribution. The data we want to fit the model to are observed grades in a macroeconomis class. 

The mechanics of the estimation is exactly the same for more complicated models. Here, model is defined by a function that can take parameters (here the mean, variance and lower_cutoff and upper_cutoff) and  and return a bunch of moments (mean, variance, soft_min and soft_max of simulated exam points). 

In [1]:
import numpy as np
import pandas as pd

from estimagic.config import EXAMPLE_DIR

## Load data

In [2]:
data = pd.read_csv(EXAMPLE_DIR / "exam_points.csv")
data.head()

,points
0,275.50
1,351.50
2,346.25
3,228.25
4,108.25


## Define function to calculate moments

Deciding which moments to use in the estimation is the most difficult part of any MSM estimation. 

Below we list the parameters we want to estimate and moments we hope are informative to identify those parameters:

|Moment           | Parameters                   |
|-----------------|------------------------------|
| mean            | mean                         |
| sd              | sd                           |
| min             | lower                        |
| max             | upper                        |

Note that such a direct equivalence of moments and parameters is not strictly needed, but it is a good way to think about the problem:

We can also estimate a version where we only use the first two moments or we could include even more moments (like shares that fall into a certain bin of points, as in Rick's original example. In general, more moments are better. 

In [3]:
def calculate_moments(sample, targets="all"):
    points = sample["points"]
    moments = {
        "mean": sample.mean()[0],
        "sd": sample.std()[0],
        "min": sample.min()[0],
        "max": sample.max()[0],
    }
    if targets != "all":
        moments = {k: v for k, v in moments.items() if k in targets}
    moments = pd.Series(moments)
    return moments

In [4]:
empirical_moments = calculate_moments(data)
empirical_moments

mean    341.908696
sd       88.752027
min      17.000000
max     449.800000
dtype: float64

## Calculate the covariance matrix of empirical moments

The covariance matrix of the empirical moments (``moments_cov``) is needed for three things:
1. to calculate the weighting matrix
2. to calculate standard errors
3. to calculate sensitivity measures

We will calculate ``moments_cov`` via a bootstrap. Depending on your problem there can be other ways to do it.

In [5]:
from estimagic import get_moments_cov

In [6]:
moments_cov = get_moments_cov(
    data, calculate_moments, bootstrap_kwargs={"n_draws": 5_000, "seed": 1234}
)
moments_cov

,mean,sd,min,max
mean,48.694552,-41.112107,24.627560,2.777834
sd,-41.112107,57.659832,-44.643781,0.739873
min,24.627560,-44.643781,226.374483,-0.068980
max,2.777834,0.739873,-0.068980,12.050210


``get_moments_cov`` mainly just calls estimagic's bootstrap function. See our [bootstrap_tutorial](../how_to_guides/inference/how_to_do_bootstrap_inference.ipynb) for background information. 



## Define a function to calculate simulated moments

In real application, this is the step that takes most of the time. However, in our very simple case, all the work is already done by scipy.

To test our function let's first set up a parameter vector (which will also serve as start parameters for the numerical optimization). 

In [7]:
params = pd.DataFrame(
    [500, 100, 0, 450],
    index=["mean", "sd", "lower", "upper"],
    columns=["value"],
)
params["lower_bound"] = [-np.inf, 0, -np.inf, -np.inf]
params

,value,lower_bound
mean,500,-inf
sd,100,0.0
lower,0,-inf
upper,450,-inf


In [8]:
from scipy.stats import truncnorm

In [9]:
def simulate_moments(params, n_draws=1_000, seed=5471):
    np.random.seed(seed)
    pardict = params["value"].to_dict()
    draws = truncnorm.rvs(
        a=(pardict["lower"] - pardict["mean"]) / pardict["sd"],
        b=(pardict["upper"] - pardict["mean"]) / pardict["sd"],
        loc=pardict["mean"],
        scale=pardict["sd"],
        size=n_draws,
    )
    sim_data = pd.DataFrame()
    sim_data["points"] = draws

    sim_moments = calculate_moments(sim_data)

    return sim_moments

In [10]:
simulate_moments(params)

mean    384.237035
sd       53.591526
min      66.218431
max     449.943485
dtype: float64

## Estimate the model parameters

Estimating a model means entails the following steps:

- Building a criterion function that measures a distance between simulated and empirical moments
- Minimizing this criterion function.
- Calculating the jacobian of the model
- Calculating standard errors, confidence intervals and p_values
- Calculating sensitivity measures

This can all be done in one go with the ``estimate_msm`` function. This function has good default values, so you only need a minimum number of inputs. However you can configure almost every aspect of the workflow via optional arguments. If you need even more control, you can call the low level functions ``estimate_msm`` is built on directly. 

In [11]:
from estimagic import estimate_msm

In [12]:
res = estimate_msm(
    simulate_moments,
    empirical_moments,
    moments_cov,
    params,
    minimize_options={"algorithm": "scipy_lbfgsb"},
)

In [13]:
res["summary"]

,value,standard_error,p_value,ci_lower,ci_upper,stars
mean,631.877994,247.486964,0.010461,146.803545,1116.952443,**
sd,198.079086,75.411009,0.008450,50.273507,345.884664,***
lower,16.754460,15.045578,0.260150,-12.734872,46.243792,
upper,449.887001,3.477616,0.000000,443.070873,456.703128,***


In [14]:
meas = "sensitivity_to_weighting"
res["sensitivity"][meas]

,mean,sd,min,max
mean,1.326711e-12,-1.609537e-12,1.237586e-14,-4.570191e-14
sd,9.447386e-13,-1.568353e-12,1.134898e-14,-4.594763e-14
lower,2.533289e-16,-6.935352e-16,5.395461e-15,-1.087335e-17
upper,3.056916e-16,-7.527505e-16,3.500084e-18,-1.560207e-14


## How to deal with calibrated parameters

In many MSM applications, not all parameters are estimated freely. Instead, some of them are calibrated outside of the model. Thus they are fixed during the estimation and no standard errors can be calculated for them. 

To do this in `estimate_msm`, you can use the same constraint syntax you already know from `maximize` and `minimize`. 

For example, in Rick's Tutorial, the cutoffs are fixed to 0 and 450, respectively. This can be expressed as follows:

In [15]:
constraints = [
    {"loc": "lower", "type": "fixed", "value": 0},
    {"loc": "upper", "type": "fixed", "value": 450},
]

res = estimate_msm(
    simulate_moments,
    empirical_moments,
    moments_cov,
    params,
    minimize_options={"algorithm": "scipy_lbfgsb"},
    constraints=constraints,
)

res["summary"]

,value,standard_error,p_value,ci_lower,ci_upper,stars
mean,596.284994,191.014884,0.001762,221.895821,970.674166,***
sd,186.845128,62.473769,0.002727,64.396541,309.293715,***
lower,0.000000,NaN,NaN,NaN,NaN,NaN
upper,450.000000,NaN,NaN,NaN,NaN,NaN


In [16]:
res["sensitivity"][meas]

,mean,sd,min,max
mean,-0.005439,-0.033146,0.034717,0.003867
sd,-0.004140,-0.032139,0.033450,0.002830


## What's in the result?

The result is a dictionary with the following entries:

In [17]:
res.keys()

dict_keys(['summary', 'cov', 'minimize_res', 'numdiff_info', 'jacobian', 'sensitivity'])

- summary: A DataFrame with estimated parameters, standard errors, p_values and confidence intervals. This is ideal for use with our `estimation_table` function. 
- cov: A DataFrame with the full covariance matrix of the estimated parameters. You can use this for hypothesis tests. 
- minimize_res: A dictionary with the complete output of the numerical optimization (if one was performed)
- numdiff_info: A dictionary with the complete output of the numerical differentiation (if one was performed)
- jacobian: A DataFrame with the jacobian of `simulate_moments` with respect to the free parameters. 

## Did it work?

We can compare this with Rick's Maximum Likelihood and MSM estimates. Note that for the likelihood estimates he fixed the lower and upper truncation values to 0 and 450, respectively. 

In [18]:
comparison = pd.DataFrame()
comparison["estimagic_msm"] = res["minimize_res"]["solution_params"][["value"]].round(1)
comparison["rick_mle"] = [622.3, 198.8, 0, 450]
comparison["rick_msm"] = [612.3, 197.3, 0, 450]
comparison

,estimagic_msm,rick_mle,rick_msm
mean,596.3,622.3,612.3
sd,186.8,198.8,197.3
lower,0.0,0.0,0.0
upper,450.0,450.0,450.0


Given that we use different estimation moments and do not fix the truncation parameters this is completely acceptable!